In [34]:
import pandas  as pd
import numpy as np

In [139]:
microbiome = pd.read_csv("DaniDatasets/BLCA_csv/microbiome_log_rna.csv",index_col=0)

In [36]:
relevant_genes = pd.read_csv("DaniDatasets/relevant_genes.csv")

In [37]:
relevant_genes

,Gene
0,A1CF
1,ABCG2
2,ABI1
3,ABI2
4,ABL1
...,...
2339,ZNF750
2340,ZNHIT6
2341,ZNRF3
2342,ZRSR2


In [38]:
svs = pd.read_csv("DaniDatasets/BLCA_csv/svs.csv", index_col=0)

In [39]:
svs.shape

(332, 377)

In [40]:
columns_to_drop = []
for column_name in svs.columns:
    gene1, gene2 = column_name.split('_')
    # Check if either gene is in genes_df
    if not ((relevant_genes['Gene'] == gene1).any() 
            or (relevant_genes['Gene'] == gene2).any()):
        columns_to_drop.append(column_name)

# Dropping
svs.drop(columns=columns_to_drop, inplace=True)
svs.shape

(332, 377)

In [41]:
mutations = pd.read_csv("DaniDatasets/BLCA_csv/mutations.csv",index_col=0)

In [42]:
mutations.shape

(332, 2030)

In [43]:
columns_to_drop = []
for column_name in mutations.columns:
    if not (relevant_genes['Gene'] == column_name).any():
        columns_to_drop.append(column_name)

## drop but is emptu

In [44]:
alterations = pd.concat([svs, mutations], axis=1)

In [45]:
alterations.shape

(332, 2407)

In [46]:
# check for index naming
for i in range(0,len(svs.index)):
    if svs.index[i] != mutations.index[i]:
        print("upsi")
print("done")

done


In [ ]:
rppa = pd.read_csv("DaniDatasets/BLCA_csv/rppa_levels.csv", index_col=0)

In [47]:
metilation = pd.read_csv("DaniDatasets/BLCA_csv/methylations.csv", index_col=0)

In [48]:
metilation[:3]

,A1CF,ABCG2,ABI1,ABI2,ABL1,ABL2,ACHE,ACSL3,ACSL6,ACTN4,...,ZMYND10,ZMYND11,ZNF133,ZNF185,ZNF331,ZNF384,ZNF703,ZNHIT6,ZRSR2,ZYX
TCGA-2F-A9KO,0.726448,0.127860,0.032748,0.039251,0.066177,0.147715,0.080597,0.031830,0.374255,0.037395,...,0.076084,0.037052,0.084956,0.666493,0.039928,0.012810,0.065221,0.358165,0.055758,0.056239
TCGA-2F-A9KQ,0.133424,0.045561,0.037451,0.033727,0.052443,0.073702,0.047542,0.027905,0.397117,0.036226,...,0.034093,0.042300,0.089750,0.635203,0.032774,0.013571,0.049114,0.069850,0.050500,0.079233
TCGA-2F-A9KR,0.349723,0.046350,0.045814,0.035994,0.043525,0.078798,0.029806,0.036979,0.352200,0.039548,...,0.028255,0.036260,0.056214,0.235256,0.041166,0.013204,0.040686,0.123200,0.057240,0.059637


In [49]:
columns_to_drop = []
for column_name in metilation.columns:
    if not (relevant_genes['Gene'] == column_name).any():
        columns_to_drop.append(column_name)

In [50]:
# drop but it's empty

In [51]:
rna_median = pd.read_csv("DaniDatasets/BLCA_csv/RNA_expression_median.csv", index_col=0)

In [52]:
columns_to_drop = []
for column_name in rna_median.columns:
    if not (relevant_genes['Gene'] == column_name).any():
        columns_to_drop.append(column_name)

In [53]:
# drop but it's empty

In [54]:
cna_log2 = pd.read_csv("DaniDatasets/BLCA_csv/cna_log2.csv", index_col=0)

In [55]:
columns_to_drop = []
for column_name in cna_log2.columns:
    if not (relevant_genes['Gene'] == column_name).any():
        columns_to_drop.append(column_name)

In [61]:
clinical_total = pd.read_csv("DaniDatasets/BLCA_csv/clinical_data.csv", index_col=1)

In [92]:
clinical_total

,Unnamed: 0,AGE,GENDER,OS_STATUS,OS_MONTHS,DSS_STATUS,DSS_MONTHS,DFS_STATUS,DFS_MONTHS,PFS_STATUS,PFS_MONTHS,CANCER_TYPE,CANCER_TYPE_DETAILED,TUMOR_TYPE,SUBTYPE
PATIENT_ID,,,,,,,,,,,,,,,
TCGA-2F-A9KO,1,63,Male,1:DECEASED,24.131242,1:DEAD WITH TUMOR,24.131242,NaN,NaN,1:PROGRESSION,21.402505,Bladder Cancer,Bladder Urothelial Carcinoma,Muscle Invasive Urothelial Carcinoma (PT2 or A...,BLCA
TCGA-2F-A9KQ,2,69,Male,0:LIVING,94.881152,0:ALIVE OR DEAD TUMOR FREE,94.881152,0:DiseaseFree,94.881152,0:CENSORED,94.881152,Bladder Cancer,Bladder Urothelial Carcinoma,Muscle Invasive Urothelial Carcinoma (PT2 or A...,BLCA
TCGA-2F-A9KR,3,59,Female,1:DECEASED,104.645428,1:DEAD WITH TUMOR,104.645428,1:Recurred/Progressed,101.456422,1:PROGRESSION,101.456422,Bladder Cancer,Bladder Urothelial Carcinoma,Muscle Invasive Urothelial Carcinoma (PT2 or A...,BLCA
TCGA-2F-A9KT,4,83,Male,0:LIVING,108.952231,0:ALIVE OR DEAD TUMOR FREE,108.952231,0:DiseaseFree,108.952231,0:CENSORED,108.952231,Bladder Cancer,Bladder Urothelial Carcinoma,Muscle Invasive Urothelial Carcinoma (PT2 or A...,BLCA
TCGA-4Z-AA7M,5,65,Male,0:LIVING,16.273794,0:ALIVE OR DEAD TUMOR FREE,16.273794,0:DiseaseFree,16.273794,0:CENSORED,16.273794,Bladder Cancer,Bladder Urothelial Carcinoma,Muscle Invasive Urothelial Carcinoma (PT2 or A...,BLCA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-ZF-AA53,328,60,Male,0:LIVING,57.895256,0:ALIVE OR DEAD TUMOR FREE,57.895256,0:DiseaseFree,57.895256,0:CENSORED,57.895256,Bladder Cancer,Bladder Urothelial Carcinoma,Muscle Invasive Urothelial Carcinoma (PT2 or A...,BLCA
TCGA-ZF-AA54,329,71,Male,1:DECEASED,19.397048,NaN,19.397048,NaN,NaN,0:CENSORED,19.397048,Bladder Cancer,Bladder Urothelial Carcinoma,Muscle Invasive Urothelial Carcinoma (PT2 or A...,BLCA
TCGA-ZF-AA56,330,79,Female,1:DECEASED,8.514975,1:DEAD WITH TUMOR,8.514975,NaN,NaN,1:PROGRESSION,8.153335,Bladder Cancer,Bladder Urothelial Carcinoma,Muscle Invasive Urothelial Carcinoma (PT2 or A...,BLCA


In [78]:
clinical = clinical_total[['AGE', 'GENDER', 'OS_STATUS', 'OS_MONTHS']]

In [79]:
def convert_status(status):
    if status == '1:DECEASED':
        return 1
    elif status == '0:LIVING':
        return 0
    else:
        print("ERROR: y cannot be empty")
        return np.NaN

clinical['OS_STATUS'] = clinical['OS_STATUS'].apply(convert_status)
clinical['OS_STATUS'] = pd.to_numeric(clinical['OS_STATUS'])

/tmp/ipykernel_54264/2028644419.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clinical['OS_STATUS'] = clinical['OS_STATUS'].apply(convert_status)
/tmp/ipykernel_54264/2028644419.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clinical['OS_STATUS'] = pd.to_numeric(clinical['OS_STATUS'])


In [148]:
# remove patients wit more than 80 yo and less than 6 months OS
filtered_clinical = clinical[~((clinical['AGE'] >= 80) & (clinical['OS_MONTHS'] <= 6))]

In [149]:
len(filtered_clinical)

320

In [150]:
len(clinical) - len(clinical[((clinical['AGE'] >= 80) & (clinical['OS_MONTHS'] <= 6))])

320

In [151]:
len(clinical)

332

In [155]:
print(microbiome.shape, filtered_clinical.shape)

(332, 1406) (320, 4)


In [156]:
merged = filtered_clinical.merge(microbiome, left_index=True, right_index=True)
merged.shape

(320, 1410)

In [160]:
merged.merge(alterations, left_index=True, right_index=True)

(320, 3817)

In [161]:
def merge_all( omics ):
    df = omics[0]
    starting_indices = [0]
    for omic in omics[1:]:
        starting_indices.append[start[-1] + omic.shape[1]]
        df.merge(omic, left_index=True, right_index=True)
    return (df, starting_indices)

In [ ]:
merge_all([filtered_clinical, rna_median, alterations, ])